In [44]:
from brax import envs
from prefacc.training.agents.prefppo import train as prefppo

In [45]:
env = envs.get_environment('inverted_double_pendulum', backend='positional')

In [46]:
# ant
# make_inference_fn, params, metrics = prefppo.train(
#   env,
#   num_timesteps=50_000_000, 
#   num_evals=10, 
#   reward_scaling=10, 
#   episode_length=1000, 
#   normalize_observations=True, 
#   action_repeat=1, 
#   unroll_length=5, 
#   num_minibatches=32, 
#   num_updates_per_batch=4, 
#   discounting=0.97, 
#   learning_rate=3e-4, 
#   entropy_cost=1e-2, 
#   num_envs=4096, 
#   batch_size=2048, 
#   seed=1,
#   num_prefs=20000)

# pendulum
# make_inference_fn, params, metrics = prefppo.train(
#         env,
#         num_timesteps=3_000_000, num_evals=20, reward_scaling=10, episode_length=1000, normalize_observations=True, action_repeat=1, unroll_length=5, num_minibatches=32, num_updates_per_batch=4, discounting=0.97, learning_rate=3e-4, entropy_cost=1e-2, num_envs=2048, batch_size=1024, seed=1, num_prefs=20000
#     )

# double_pendulum
make_inference_fn, params, metrics = prefppo.train(
        env,
        num_timesteps=30_000_000, num_evals=20, reward_scaling=10, episode_length=1000, normalize_observations=True, action_repeat=1, unroll_length=5, num_minibatches=32, num_updates_per_batch=4, discounting=0.97, learning_rate=3e-4, entropy_cost=1e-2, num_envs=2048, batch_size=1024, seed=1
    )


#walker 2d
# make_inference_fn, params, metrics = prefppo.train(
#           env,
#   num_timesteps=10_000_000, 
#   num_evals=20, 
#   reward_scaling=10, 
#   episode_length=1000, 
#   normalize_observations=True, 
#   action_repeat=1, 
#   unroll_length=5, 
#   num_minibatches=32, 
#   num_updates_per_batch=4, 
#   discounting=0.97, 
#   learning_rate=3e-4, 
#   entropy_cost=1e-2, 
#   num_envs=4096, 
#   batch_size=2048, 
#   seed=1,
#   num_prefs=4000)

print("Training complete")

Training complete


In [47]:
print(metrics)

{'eval/walltime': 14.328125476837158, 'training/sps': np.float64(1087562.190466951), 'training/walltime': 37.305792808532715, 'training/entropy_loss': Array(0.0133847, dtype=float32), 'training/policy_loss': Array(-0.00286026, dtype=float32), 'training/total_loss': Array(0.46769878, dtype=float32), 'training/v_loss': Array(0.45717436, dtype=float32), 'pearson_correlation': Array([-0.31297392], dtype=float32), 'eval/episode_reward': Array(4281.054, dtype=float32), 'eval/episode_reward_std': Array(3757.3096, dtype=float32), 'eval/avg_episode_length': Array(462.78125, dtype=float32), 'eval/epoch_eval_time': 0.4318830966949463, 'eval/sps': 296376.49859311525}


In [48]:
from brax.io import model
from brax.io import json
from brax.io import html

In [49]:
model.save_params('/tmp/params', params)

In [50]:
params = model.load_params('/tmp/params')
inference_fn = make_inference_fn(params)

In [51]:
env = envs.create('inverted_double_pendulum', backend='positional')

In [52]:
import jax

In [53]:
jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
jit_inference_fn = jax.jit(inference_fn)

In [54]:
rollout = []
rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(rng=rng)
for _ in range(1000):
  rollout.append(state.pipeline_state)
  act_rng, rng = jax.random.split(rng)
  act, _ = jit_inference_fn(state.obs, act_rng)
  state = jit_env_step(state, act)

In [55]:
from IPython.display import HTML, clear_output

In [56]:
HTML(html.render(env.sys.tree_replace({'opt.timestep': env.dt}), rollout))